In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Load the dataset
df = pd.read_excel("messy_marketing_campaign.xlsx")

# Step 2: Standardize column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Step 3: Fix data types
# Handle 'income' (replace 'unknown' and empty strings, convert to numeric)
df['income'] = df['income'].replace(['unknown', ''], pd.NA)
df['income'] = pd.to_numeric(df['income'], errors='coerce')

# Standardize 'dt_customer' to datetime
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, errors='coerce')
    except:
        return pd.to_datetime(date_str, format='%d-%m-%Y', errors='coerce')

df['dt_customer'] = df['dt_customer'].apply(parse_date)

# Ensure numerical columns are numeric
numerical_cols = ['year_birth', 'kidhome', 'teenhome', 'recency', 'mntwines', 'mntfruits',
                  'mntmeatproducts', 'mntfishproducts', 'mntsweetproducts', 'mntgoldprods',
                  'numdealspurchases', 'numwebpurchases', 'numcatalogpurchases',
                  'numstorepurchases', 'numwebvisitsmonth', 'acceptedcmp3', 'acceptedcmp4',
                  'acceptedcmp5', 'acceptedcmp1', 'acceptedcmp2', 'complain',
                  'z_costcontact', 'z_revenue', 'response']
df[numerical_cols] = df[numerical_cols].apply(pd.to_numeric, errors='coerce')

# Convert categorical columns to string
df[['education', 'marital_status']] = df[['education', 'marital_status']].astype(str)

# Step 4: Standardize categorical values
df['education'] = df['education'].str.title()
df['marital_status'] = df['marital_status'].str.title()
marital_status_mapping = {'Yolo': 'Unknown', 'Absurd': 'Unknown', 'Alone': 'Single'}
df['marital_status'] = df['marital_status'].replace(marital_status_mapping)

# Step 5: Handle missing values
df['income'].fillna(df['income'].median(), inplace=True)
df['recency'].fillna(df['recency'].median(), inplace=True)
df['dt_customer'].fillna(df['dt_customer'].median(), inplace=True)

# Step 6: Handle outliers
# Cap 'income' and 'mntmeatproducts' at 99th percentile
df['income'] = df['income'].clip(upper=df['income'].quantile(0.99))
df['mntmeatproducts'] = df['mntmeatproducts'].clip(upper=df['mntmeatproducts'].quantile(0.99))

# Step 7: Remove duplicates
df = df.drop_duplicates().drop_duplicates(subset='id', keep='first')

# Step 8: Validate data consistency
# Remove negative values (if any)
df[numerical_cols] = df[numerical_cols].clip(lower=0)
# Filter invalid 'year_birth' (before 1900 or after 2000)
df = df[(df['year_birth'] >= 1900) & (df['year_birth'] <= 2000)]
# Filter future 'dt_customer' dates
df = df[df['dt_customer'] <= '2025-04-21']
# Drop constant columns
df = df.drop(['z_costcontact', 'z_revenue'], axis=1)

# Step 9: Save cleaned dataset
df.to_excel("cleaned_marketing_campaign.xlsx", index=False)

# Print summary
print("Cleaning Complete. Summary:")
print(f"Rows after cleaning: {len(df)}")
print(f"Columns after cleaning: {len(df.columns)}")
print("Missing Values:\n", df.isnull().sum())
print("Data Types:\n", df.dtypes)
print("Cleaned dataset saved as 'cleaned_marketing_campaign.xlsx'")

C:\Users\Vinit\AppData\Local\Temp\ipykernel_22184\811059838.py:19: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(date_str, errors='coerce')
C:\Users\Vinit\AppData\Local\Temp\ipykernel_22184\811059838.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['income'].fillna(df['income'].median(), inplace=True)
C:\Users\Vinit\AppData\Local\Temp\ipykernel_22184\811059838.py:45: FutureWarning: A

Cleaning Complete. Summary:
Rows after cleaning: 2238
Columns after cleaning: 27
Missing Values:
 id                     0
year_birth             0
education              0
marital_status         0
income                 0
kidhome                0
teenhome               0
dt_customer            0
recency                0
mntwines               0
mntfruits              0
mntmeatproducts        0
mntfishproducts        0
mntsweetproducts       0
mntgoldprods           0
numdealspurchases      0
numwebpurchases        0
numcatalogpurchases    0
numstorepurchases      0
numwebvisitsmonth      0
acceptedcmp3           0
acceptedcmp4           0
acceptedcmp5           0
acceptedcmp1           0
acceptedcmp2           0
complain               0
response               0
dtype: int64
Data Types:
 id                              int64
year_birth                      int64
education                      object
marital_status                 object
income                        float64
kidhome    